In [1]:
class Range:
    def __init__(self, vals): self.min, self.max = min(vals), max(vals)
    def __contains__(self, val): return self.min <= val < self.max
    def __str__(self): return "%.3f %.3f"%(self.min, self.max)

class Point:
    __str__ = lambda self: "Point(%.3f, %.3f)"%(self.x, self.y) 
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Line:
    def __init__(self, start_point, end_point):
        self.start  = start_point
        self.end    = end_point
        self.y_bar  = start_point.x == end_point.x
        self.x_bar  = start_point.y == end_point.y
        if not self.y_bar:
            self.slope  = self.slope()
            self.intercept = self.end.y - self.slope*self.end.x
        self.xrange = Range([start_point.x, end_point.x])
        self.yrange = Range([start_point.y, end_point.y])
    
    def slope(self):
        return (self.end.y - self.start.y)/(self.end.x - self.start.x)
    
    def __getitem__(self, i): return [self.start, self.end][i]
    def __str__(self): return "Line(%s,%s)" %(str(self.start), str(self.end))
    
    def intersects(self, other):
        if self.x_bar:
            if other.x_bar: return False
            if other.y_bar: return self.end.y in other.yrange and other.end.x in self.xrange
            xp = (self.end.y-other.intercept)/other.slope
            return xp in self.xrange and xp in other.xrange and self.end.y in other.yrange
        if self.y_bar:
            if other.x_bar: return other.end.y in self.yrange and self.end.x in other.xrange
            if other.y_bar: return False
            yp = (other.slope * self.end.x + other.intercept)
            return yp in self.yrange and yp in other.yrange and self.end.x in other.xrange

        if other.x_bar:
            xp = (other.end.y-self.intercept)/self.slope
            return xp in other.xrange and xp in self.xrange and other.end.y in self.yrange
        if other.y_bar:
            yp = (self.slope * other.end.x + self.intercept)
            return yp in other.yrange and other.end.x in self.xrange and yp in self.yrange
        
        if self.slope == other.slope: return False
        xp = (other.intercept - self.intercept) / (self.slope - other.slope)
        yp = self.slope * xp + self.intercept

        return xp in self.xrange and xp in other.xrange and \
                yp in self.yrange and yp in other.yrange
    
    def point_on_line(self, point):
        # Point on line with tolerance
        if self.x_bar: return point.y==self.end.y and point.x in self.xrange
        if self.y_bar: return point.x==self.end.x and point.y in self.yrange
        return abs(point.y - self.slope * point.x - self.intercept) < 0.001

class Obstacle:
    def __init__(self, vertices):
        self.x = Range([v[0] for v in vertices])
        self.y = Range([v[1] for v in vertices])
        
        self.lines = [Line(Point(*vertices[i]), Point(*vertices[i+1])) for i in range(len(vertices)-1)]
    
    def check_collisions(self, line, verbose=False):
        if self.point_in_obstacle(line[0]) or self.point_in_obstacle(line[1]):
            return True
        
        if line.xrange.max < self.x.min or line.xrange.min > self.x.max or \
            line.yrange.max < self.y.min or line.yrange.min > self.y.max:
            return False
        
        for l in self.lines:
            if l.intersects(line): return True
        return False
    
    def point_in_obstacle(self, pt, verbose=False):
        # Sanity check, point is in obstacle range
        if not (self.x.min <= pt.x <= self.x.max and self.y.min <= pt.y <= self.y.max):
            return False
        
        projection = Line(pt, Point(self.x.max+5, pt.y))
        num_cross  = 0
        
        for l in self.lines:
            # Exception, if point is in line (very rare) return true
            if l.point_on_line(pt): return True
            num_cross += l.intersects(projection)
            
        return bool(num_cross % 2)

class Obstacles:
    """ List of obstacles """
    def __init__(self, obstacles):
        self.obss = [Obstacle([tuple(pt.astype(int)) for pt in vertices]) for vertices in obstacles]
    
    def point_is_valid(self, x, y):
        """ Returns whether or not q=(x,y) \in Q_free """
        point = Point(x, y)
        for obs in self.obss:
            if obs.point_in_obstacle(point): return False
        return True

    def check_collisions(self, line, verbose=False):
        line = Line(Point(*line[0]), Point(*line[1]))
        
        for obs in self.obss:
            if obs.check_collisions(line, verbose):
                return True
        return False

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.lines import Line2D
import matplotlib.patches as patches
from collections import namedtuple
import numpy as np
import random, math
from shapely.geometry import Polygon, LineString
from shapely.geometry import Point as SPoint

from importlib import reload
import KDTree
KDTree = reload(KDTree)

In [2]:
def get_obstacle_course(obstacle_path):
    vertices = list()
    codes = [Path.MOVETO]
    with open(obstacle_path) as f:
        quantity = int(f.readline())
        lines = 0
        for line in f:
            coordinates = tuple(map(int, line.strip().split(' ')))
            if len(coordinates) == 1:
                codes += [Path.MOVETO] + [Path.LINETO]*(coordinates[0]-1) + [Path.CLOSEPOLY]
                vertices.append((0,0)) #Always ignored by closepoly command
            else:
                vertices.append(coordinates)
    vertices.append((0,0))
    vertices = np.array(vertices, float)
    path = Path(vertices, codes)
    return path

def get_start_and_goal(start_goal_path):
    start, goal = None, None
    with open(start_goal_path) as f:
        start = tuple(map(int, f.readline().strip().split(' ')))
        goal  = tuple(map(int, f.readline().strip().split(' ')))
    
    return start, goal

def draw_obstacle_course(path, ax):
    pathpatch = patches.PathPatch(path, facecolor='None', edgecolor='xkcd:violet')
    
    ax.add_patch(pathpatch)
    ax.set_title('Rapidly-exploring Random Tree')

    ax.dataLim.update_from_data_xy(path.vertices)
    ax.autoscale_view()
    ax.invert_yaxis()

def draw_start_and_goal(start, goal, ax):
    ax.add_patch(patches.Circle(start, facecolor='xkcd:bright green', zorder=10))
    ax.add_patch(patches.Circle(goal, facecolor='xkcd:fuchsia', zorder=10))

def gen_next(q_near, q_rand, scale=1):
    vec = q_rand - q_near
    return tuple(q_near + vec * scale / np.linalg.norm(vec))

def shapely_intersect(line):
    line = LineString(line)
    for shape in shapes:
        if line.intersects(shape): return True
    return False

def shapely_contains(point):
    point = SPoint(*point)
    for shape in shapes:
        if shape.contains(point) or shape.boundary.contains(point): return True
    return False

def gen_rand(low, high, size):
    arrs = []
    for i in range(len(low)):
        span = high[i]-low[i]
        arrs.append((np.random.random(size)*span*1.1 + low[i]-span*0.05).reshape(-1,1))
    return np.concatenate(arrs, 1)

In [ ]:
class Range:
    def __init__(self, vals): self.min, self.max = min(vals), max(vals)
    def __contains__(self, val): return self.min <= val < self.max
    def __str__(self): return "%.3f %.3f"%(self.min, self.max)

In [ ]:
class Point:
    __str__ = lambda self: "Point(%.3f, %.3f)"%(self.x, self.y) 
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Line:
    def __init__(self, start_point, end_point):
        self.start  = start_point
        self.end    = end_point
        self.y_bar  = start_point.x == end_point.x
        self.x_bar  = start_point.y == end_point.y
        if not self.y_bar:
            self.slope  = self.slope()
            self.intercept = self.end.y - self.slope*self.end.x
        self.xrange = Range([start_point.x, end_point.x])
        self.yrange = Range([start_point.y, end_point.y])
    
    def slope(self):
        return (self.end.y - self.start.y)/(self.end.x - self.start.x)
    
    def __getitem__(self, i): return [self.start, self.end][i]
    def __str__(self): return "Line(%s,%s)" %(str(self.start), str(self.end))
    
    def intersects(self, other):
#         print(self.x_bar, self.y_bar)
        if self.x_bar:
            if other.x_bar: return False
            if other.y_bar: return self.end.y in other.yrange and other.end.x in self.xrange
            xp = (self.end.y-other.intercept)/other.slope
#             print(xp, self.end.y)
            return xp in self.xrange and xp in other.xrange and self.end.y in other.yrange
        if self.y_bar:
            if other.x_bar: return other.end.y in self.yrange and self.end.x in other.xrange
            if other.y_bar: return False
            yp = (other.slope * self.end.x + other.intercept)
            return yp in self.yrange and yp in other.yrange and self.end.x in other.xrange

        if other.x_bar:
            xp = (other.end.y-self.intercept)/self.slope
#             print(xp, other.end.y)
            return xp in other.xrange and xp in self.xrange and other.end.y in self.yrange
        if other.y_bar:
            yp = (self.slope * other.end.x + self.intercept)
            return yp in other.yrange and other.end.x in self.xrange and yp in self.yrange
        
        if self.slope == other.slope: return False
        xp = (other.intercept - self.intercept) / (self.slope - other.slope)
        yp = self.slope * xp + self.intercept

        return xp in self.xrange and xp in other.xrange and \
                yp in self.yrange and yp in other.yrange
    
    def point_on_line(self, point):
        # Point on line with tolerance
        if self.x_bar: return point.y==self.end.y and point.x in self.xrange
        if self.y_bar: return point.x==self.end.x and point.y in self.yrange
        return abs(point.y - self.slope * point.x - self.intercept) < 0.001

In [ ]:
class Obstacles:
    """ List of obstacles """
    def __init__(self, obstacles):
        self.obss = [Obstacle([tuple(pt.astype(int)) for pt in vertices]) for vertices in obstacles]
    
    def point_is_valid(self, x, y):
        """ Returns whether or not q=(x,y) \in Q_free """
        point = Point(x, y)
        for obs in self.obss:
            if obs.point_in_obstacle(point): return False
        return True

    def check_collisions(self, line, verbose=False):
        line = Line(Point(*line[0]), Point(*line[1]))
        
        for obs in self.obss:
            if obs.check_collisions(line, verbose):
                return True
        return False

class Obstacle:
    def __init__(self, vertices):
        self.x = Range([v[0] for v in vertices])
        self.y = Range([v[1] for v in vertices])
        
        self.lines = [Line(Point(*vertices[i]), Point(*vertices[i+1])) for i in range(len(vertices)-1)]
    
    def check_collisions(self, line, verbose=False):
        if self.point_in_obstacle(line[0]) or self.point_in_obstacle(line[1]):
            return True
        
        if line.xrange.max < self.x.min or line.xrange.min > self.x.max or \
            line.yrange.max < self.y.min or line.yrange.min > self.y.max:
            return False
        
        for l in self.lines:
            if l.intersects(line): return True
        return False
    
    def point_in_obstacle(self, pt, verbose=False):
        # Sanity check, point is in obstacle range
        if not (self.x.min <= pt.x <= self.x.max and self.y.min <= pt.y <= self.y.max):
            return False
        
        projection = Line(pt, Point(self.x.max+5, pt.y))
        num_cross  = 0
        
        for l in self.lines:
            # Exception, if point is in line (very rare) return true
            if l.point_on_line(pt): return True
            num_cross += l.intersects(projection)
            
        return bool(num_cross % 2)

In [ ]:
obstacle_path, goal_path = "../hw3/world_obstacles.txt", "../hw3/goal.txt"
# obstacle_path, goal_path = "world_obstacles.txt", "start_goal.txt"
path = get_obstacle_course(obstacle_path)
start, goal = get_start_and_goal(goal_path)

obstacles = Obstacles(path.to_polygons())
shapes = [Polygon(polyg) for polyg in path.to_polygons()]
num_tests = 10000

In [ ]:
""" Unit test for line in obstacle """
minp, maxp = path.vertices.min(0), path.vertices.max(0)
step_size = 50

starts = gen_rand(minp, maxp, num_tests)
rands  = gen_rand(minp, maxp, num_tests)

# starts, rands = starts.astype(int), rands.astype(int)

dirs  = starts - rands
ends  = starts + dirs / np.hypot(dirs[:,0], dirs[:,1])[:,None] * step_size
paths = [Path([starts[i], ends[i]], [Path.MOVETO, Path.LINETO]) for i in range(num_tests)]

# Shapely answer
my_ans = [obstacles.check_collisions(paths[i].vertices) for i in range(num_tests)]
ex_ans = [shapely_intersect(paths[i].vertices) for i in range(num_tests)]

correct_positives, correct_negatives = [], []
false_positives, false_negatives = [], []
for i in range(num_tests):
    if   my_ans[i] and ex_ans[i]:         correct_positives.append(i)
    elif not my_ans[i] and not ex_ans[i]: correct_negatives.append(i)
    elif ex_ans[i] and not my_ans[i]:     false_negatives.append(i)
    elif my_ans[i] and not ex_ans[i]:     false_positives.append(i)
        
print("Correct : ", len(correct_positives), len(correct_negatives))
print("False   : ", len(false_positives), len(false_negatives))

In [ ]:
for idx in false_negatives:
    print(idx, "Line(Point(%.3f,%.3f),Point(%.3f,%.3f))"%tuple(paths[idx].vertices.reshape(-1)), my_ans[idx], ex_ans[idx])

In [ ]:
test_point = 7084
test_path = paths[test_point].vertices
obs_path  = Line(Point(*test_path[0]), Point(*test_path[1]))
shp_path  = LineString(test_path)

found = None
for idx in range(len(obstacles.obss)):
    if obstacles.obss[idx].check_collisions(obs_path) != shapes[idx].intersects(shp_path):
        found = idx
        break

if found is not None:
    idx = found
    print(idx, obs_path, obstacles.check_collisions(test_path))

    obstacle = obstacles.obss[idx]    
    for l in obstacle.lines:
        print(l, l.intersects(obs_path))

shapes[found]

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots()
draw_obstacle_course(path, ax)
draw_start_and_goal(start, goal, ax)
plt.show()

axtext = lambda x, y: "(%d, %d)"%(x,y)
    
for (x,y) in path.vertices:
    if x == 0 and y == 0: continue
    ax.text(x, y, axtext(x, y), fontsize=8, color='xkcd:purple')
    
# for idx in correct_positives:
#     ax.add_patch(patches.PathPatch(paths[idx], color='xkcd:red', lw=1))

# for idx in correct_negatives:
#     ax.add_patch(patches.PathPatch(paths[idx], color='xkcd:green', lw=1))

for idx in false_negatives:
    ax.add_patch(patches.PathPatch(paths[idx], color='xkcd:red', lw=2))

for idx in false_positives:
    ax.add_patch(patches.PathPatch(paths[idx], color='xkcd:green', lw=3))

plt.show()

In [ ]:
""" Unit test for point in obstacle """
minp, maxp = path.vertices.min(0), path.vertices.max(0)
points = gen_rand(minp, maxp, num_tests)

# points = points.astype(int)

my_ans = [not obstacles.point_is_valid(*points[i]) for i in range(num_tests)]
ex_ans = [shapely_contains(points[i]) for i in range(num_tests)]

correct_positives, correct_negatives = [], []
false_positives, false_negatives = [], []
for i in range(num_tests):
    if   my_ans[i] and ex_ans[i]:         correct_positives.append(i)
    elif not my_ans[i] and not ex_ans[i]: correct_negatives.append(i)
    elif ex_ans[i] and not my_ans[i]:     false_negatives.append(i)
    elif my_ans[i] and not ex_ans[i]:     false_positives.append(i)
        
print("Correct : ", len(correct_positives), len(correct_negatives))
print("False   : ", len(false_positives), len(false_negatives))

In [ ]:
for i in false_negatives:
    print(points[i], my_ans[i], ex_ans[i])

In [ ]:
test_point = (11.000, 452.000)

pt, spt = Point(*test_point), SPoint(*test_point)
found = None
for idx in range(len(obstacles.obss)):
    if obstacles.obss[idx].point_in_obstacle(pt) != shapes[idx].contains(spt):
        found = idx
        break

if found is not None:
    idx = found
    print(idx)
    obstacle = obstacles.obss[idx]
    projection = Line(pt, Point(obstacle.x.max+5, pt.y))
    print(projection)
    num_cross  = 0
    for l in obstacle.lines:
        print(l, l.intersects(projection), projection.intersects(l))
        # Exception, if point is in line (very rare) return true
        if l.point_on_line(pt): print(True); break
        num_cross += l.intersects(projection)
    
    print(num_cross, bool(num_cross % 2), shapes[idx].contains(spt))

    shapes[idx]

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots()
draw_obstacle_course(path, ax)
draw_start_and_goal(start, goal, ax)
plt.show()

axtext = lambda x, y: "(%d, %d)"%(x,y)
    
for node in path.vertices:
    x, y = node
    if x == 0 and y == 0: continue
    ax.text(x, y, axtext(x, y), fontsize=5, color='xkcd:purple')

for idx in correct_positives:
    ax.add_patch(patches.Circle(points[idx], 3, color='xkcd:green'))

for idx in correct_negatives:
    ax.add_patch(patches.Circle(points[idx], 3, color='xkcd:red'))

for idx in false_negatives:
    ax.add_patch(patches.Circle(points[idx], 3, color='xkcd:red'))
#     ax.text(points[idx][0]+3, points[idx][1]-3 , axtext(points[idx][0], points[idx][1]), fontsize=5, zorder=5)

for idx in false_positives:
    ax.add_patch(patches.Circle(points[idx], 3, color='xkcd:green'))
#     ax.text(points[idx][0]+3, points[idx][1]-3 , axtext(points[idx][0], points[idx][1]), fontsize=5, zorder=5)